In [2]:
import tensorflow as tf
from imblearn.over_sampling import BorderlineSMOTE
import numpy as np
import random

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
EPOCHS = 100

In [4]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
    self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, x, training=False, mask=None):
    x = self.flatten(x)
    x = self.dense1(x)
    return self.dense2(x)

In [5]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train_small = list()
y_train_small = list()
for x, y in zip(x_train, y_train):
  if (y == 0 and random.randint(0,100) < 10) or y == 1:
    x_train_small.append(x[:])
    y_train_small.append(y)

x_test_small = list()
y_test_small = list()
for x, y in zip(x_test, y_test):
  if y == 0 or y == 1:
    x_test_small.append(x[:])
    y_test_small.append(y)

x_train = np.stack(x_train_small, axis=0)
y_train = np.stack(y_train_small, axis=0)

x_test = np.stack(x_test_small, axis=0)
y_test = np.stack(y_test_small, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

170500096/170498071 [==============================] - 5s 0us/step


In [6]:
model = MyModel()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
172/172 [==============================] - 6s 27ms/step - loss: 0.5358 - accuracy: 0.9091 - precision: 0.9211 - recall: 0.9852 - val_loss: 0.5830 - val_accuracy: 0.6890 - val_precision: 0.6215 - val_recall: 0.9670
Epoch 2/100
172/172 [==============================] - 4s 25ms/step - loss: 0.2693 - accuracy: 0.9130 - precision: 0.9213 - recall: 0.9883 - val_loss: 0.7242 - val_accuracy: 0.6630 - val_precision: 0.5996 - val_recall: 0.9810
Epoch 3/100
172/172 [==============================] - 4s 25ms/step - loss: 0.2249 - accuracy: 0.9235 - precision: 0.9339 - recall: 0.9859 - val_loss: 0.6199 - val_accuracy: 0.7025 - val_precision: 0.6309 - val_recall: 0.9760
Epoch 4/100
172/172 [==============================] - 4s 25ms/step - loss: 0.2187 - accuracy: 0.9311 - precision: 0.9375 - recall: 0.9906 - val_loss: 0.9612 - val_accuracy: 0.6320 - val_precision: 0.5769 - val_recall: 0.9900
Epoch 5/100
172/172 [==============================] - 4s 25ms/step - loss: 0.2339 - accuracy: 0